In [ ]:
!pip install emoji

In [ ]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
%cd transformers

/content/transformers


In [ ]:
!pip3 install --upgrade .

Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3179591 sha256=dc78ca06c1e16d1fc43e1e093bdc75eb39b67d05b253efe7b516540dc2ebae34
  Stored in directory: /tmp/pip-ephem-wheel-cache-5btz51cm/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.13.0.dev0
    Uninstalling trans

In [ ]:
import os
import json
import random
import time
import datetime
import torch
import argparse
import numpy as np
import pandas as pd
from torch.nn import functional as F
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix
from transformers import get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler, Dataset

In [ ]:
batch_size = 4

In [ ]:
def calculate_scores(preds, labels):
    pred_flat = np.argmax(np.concatenate(preds), axis=1).flatten()
    results = dict()
    results['precision_score'] = precision_score(labels, pred_flat, average='macro')
    results['recall_score'] = recall_score(labels, pred_flat, average='macro')
    results['f1_score'] = f1_score(labels, pred_flat, average='macro')
    results['confusion_matrix'] = confusion_matrix(labels,pred_flat)
    return results


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

Preprocessing

In [ ]:
def convert_label(label):
    if label == "desirability":
        return 0
    elif label == "enabling context":
        return 1
    elif label == "can do":
        return 2
    elif label == "buzz":
        return 3
    elif label == "invitation":
        return 4
    else:
        raise Exception("label classes must be 'desirability' or 'enabling context' or 'buzz' or 'can do' or 'invitation'")
        
        
def convert_prediction(pred):
     if pred == "desirability":
        return 0
     elif pred == "enabling context":
        return 1
     elif pred == "can do":
        return 2
     elif pred == "buzz":
        return 3
     elif pred == "invitation":
        return 4
     else:
        raise Exception("Prediction classes must be 'desirability' or 'enabling context' or 'buzz' or 'can do' or 'invitation'")

In [ ]:
%cd ..

/content


In [ ]:
df = pd.read_csv('Dataset_V4.csv')
df

,Unnamed: 0,decision,created_at,id,id_str,full_text,source,retweet_count,favorite_count,reply,username_length,user_location
0,0,enabling context,2021-01-30 02:40:25+00:00,1355345090022154241,1355345090022154241,@science_barbie @priyology Cyanobacteria (also...,Twitter Web App,0,0,True,8,Unknown
1,1,enabling context,2021-01-30 01:05:16+00:00,1355321147701686273,1355321147701686273,New insights into coral bleaching by internati...,Twitter Web App,1,1,False,10,Unknown
2,2,buzz,2021-01-29 22:49:17+00:00,1355286925809094659,1355286925809094659,"Friday Tally:\n\n2 dives, continued coral blea...",Twitter for iPhone,0,1,False,11,Unknown
3,3,desirability,2021-01-29 22:09:20+00:00,1355276872410681348,1355276872410681348,Coral bleaching is so sad because they just be...,Twitter for iPhone,0,2,False,8,Unknown
4,4,desirability,2021-01-29 21:34:44+00:00,1355268165148815360,1355268165148815360,predicts that at the current rate of emissions...,Twitter Web App,0,0,True,13,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
1217,1217,buzz,2021-05-17 19:30:12+00:00,1394374712835264513,1394374712835264513,#ClimateChampions developed learning resources...,Twitter Web App,4,14,False,10,United States
1218,1218,desirability,2021-05-17 17:37:33+00:00,1394346363736018949,1394346363736018949,"""For all we hear about #coral bleaching &amp; ...",Twitter Web App,11,24,False,14,Unknown
1219,1219,enabling context,2021-05-17 17:01:39+00:00,1394337327930122250,1394337327930122250,Corals are critical. Track coral bleaching in ...,Hootsuite Inc.,10,29,False,13,United States
1220,1220,enabling context,2021-05-17 16:09:02+00:00,1394324089200779268,1394324089200779268,The number one root cause of coral reefs facin...,Twitter for iPhone,1,0,True,12,South Korea


In [ ]:
df = df[['decision','full_text']]
df = df.rename(columns={'full_text':'text','decision':'label'})

In [ ]:
df['label'] = df['label'].map({
    'desirability': 0, 
    'enabling context': 1,
    'can do': 2,
    'buzz':3,
    'invitation': 4
    })

In [ ]:
df['label'] = df['label'].astype(int)

Split into TEST and TRAIN

---



In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in sss.split(df[['text']], df[['label']]):
  train = df.iloc[train_idx]
  test = df.iloc[test_idx]

print(len(train))
print(len(test))
val = test

977
245


In [ ]:
train_df = train.reset_index(drop=True)
dev_df = val.reset_index(drop=True)
test_df = test.reset_index(drop=True)

In [ ]:
train_df

,label,text
0,3,Thanks to David Vaughan and Chris Page for mak...
1,0,An intense marine heatwave occurred in 2016.\n...
2,4,We are looking for a new Coral Biologist at Ve...
3,4,Join me and authors @Angie_Hockman and @Holroy...
4,3,.@AMV_BBDO and @ShebaBrand's dramatic satellit...
...,...,...
972,2,"Thankfully this year, cyclones saved the coral..."
973,0,Higher temperature has contributed to coral bl...
974,0,A reminder that climate change is not happenin...
975,0,#Drought over the southwestern Tibetan Plateau...


In [ ]:
dev_df

,label,text
0,0,The waters off southern NSW and the east coast...
1,2,From coral restoration dives to cleanup dives ...
2,3,Fairmont Maldives Sirru Fen Fushi announces th...
3,1,The legislation bans the sale of over-the-coun...
4,3,Volunteer pup Mango helping to get the word ou...
...,...,...
240,0,"For example, Rising temperatures cause coral b..."
241,1,@wlbeeton @ddasser @ClimateDepot 93% of global...
242,3,Think these corals are bleached? Think again! ...
243,1,Coral Species Respond Differently To Ocean War...


In [ ]:
test_df

,label,text
0,0,The waters off southern NSW and the east coast...
1,2,From coral restoration dives to cleanup dives ...
2,3,Fairmont Maldives Sirru Fen Fushi announces th...
3,1,The legislation bans the sale of over-the-coun...
4,3,Volunteer pup Mango helping to get the word ou...
...,...,...
240,0,"For example, Rising temperatures cause coral b..."
241,1,@wlbeeton @ddasser @ClimateDepot 93% of global...
242,3,Think these corals are bleached? Think again! ...
243,1,Coral Species Respond Differently To Ocean War...


In [ ]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    for sentence in df[["text"]].values:
        sentence = sentence.item()
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      
                            add_special_tokens = True,  
                            max_length = 72,
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [ ]:
def prepare_dataloaders(train_df,test_df,dev_df, batch_size):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

    tweet_valid = bert_encode(dev_df, tokenizer)
    tweet_valid_labels = dev_df.label.astype(int)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df.label.astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)


    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)
    
    input_ids, attention_masks = tweet_valid.values()
    labels = torch.tensor(tweet_valid_labels.values)
    val_dataset = TensorDataset(input_ids, attention_masks, labels)
    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    validation_dataloader = DataLoader(
                val_dataset, 
                sampler = SequentialSampler(val_dataset),
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    
    return train_dataloader,validation_dataloader,test_dataloader

In [ ]:
train_dataloader,validation_dataloader,test_dataloader = prepare_dataloaders(train_df, test_df, dev_df, batch_size = batch_size)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Experiment

In [ ]:
def test_encode(sentence):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)

    encoded_dict = tokenizer.encode_plus(
                        sentence,                      
                        add_special_tokens = True,
                        max_length = 72,
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                )
           
    return encoded_dict['input_ids']

In [ ]:
def test_decode(tokens):
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    return tokenizer.convert_ids_to_tokens(tokens)

In [ ]:
train_df.text[0]

'Thanks to David Vaughan and Chris Page for making this possible. https://t.co/q4qaW3Lca4'

In [ ]:
text_test = train_df.text[0]
text_encoded = test_encode(text_test)
text_decoded = test_decode(text_encoded[0, :130])


print(f'Shape      : {text_encoded.shape}')
print(f'Word Ids   : {text_encoded}')
print(f'Decoded Words   : {text_decoded}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Shape      : torch.Size([1, 72])
Word Ids   : tensor([[    0,   404,     9,  1593, 42578,    13,  1687,  5810,    19,   382,
            33,  1183,     4,    10,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]])
Decoded Words   : ['<s>', 'Thanks', 'to', 'David', 'Vaughan', 'and', 'Chris', 'Page', 'for', 'making', 'this', 'possible', '.', 'HTTPURL', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

Optimizer for BERTweet model

In [ ]:
from transformers import RobertaConfig, RobertaModel, BertPreTrainedModel
from torch.nn import CrossEntropyLoss
from torch import nn
from typing import Tuple
import torch

!wget https://public.vinai.io/BERTweet_base_transformers.tar.gz

!tar -xzvf BERTweet_base_transformers.tar.gz

class BERTweetModelForClassification(BertPreTrainedModel):
    base_model_prefix = "roberta"

    def __init__(self):
        self.num_labels: int = 5
        config: RobertaConfig = RobertaConfig.from_pretrained(
            "vinai/bertweet-base",
            output_hidden_states=True,
        )
        super().__init__(config)
        self.model: RobertaModel = RobertaModel.from_pretrained(
            "vinai/bertweet-base",
            config=config
        )
        self.dense = nn.Linear(in_features=768 * 2,
                               out_features=768,
                               )
        self.dropout = nn.Dropout(p=0.1)
        self.dense_2 = nn.Linear(in_features=768,
                                 out_features=256,
                                 )
        self.classifier = nn.Linear(in_features=256,
                                    out_features=self.num_labels,
                                    )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
    ):
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
        )
        # Take <CLS> token for Native Layer Norm Backward
        hidden_states: Tuple[torch.tensor] = outputs[2]
        last_sequence_output: torch.tensor = hidden_states[-1][:, 0, :]
        second_to_last_sequence_output: torch.tensor = hidden_states[-2][:, 0, :]

        sequence_output: torch.tensor = torch.cat((
            last_sequence_output,
            second_to_last_sequence_output
        ), dim=1)

        sequence_output = self.dense(sequence_output)
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.dense_2(sequence_output)
        sequence_output = self.dropout(sequence_output)

        logits: torch.tensor = self.classifier(sequence_output)
        outputs = (logits,)
        if labels is not None:
            loss_function = CrossEntropyLoss()
            loss = loss_function(
                logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs
        return outputs  # loss, logits

--2021-11-30 06:53:24--  https://public.vinai.io/BERTweet_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 65.8.49.115, 65.8.49.107, 65.8.49.47, ...
Connecting to public.vinai.io (public.vinai.io)|65.8.49.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322076118 (307M) [application/x-tar]
Saving to: ‘BERTweet_base_transformers.tar.gz.9’

BERTweet_base_trans 100%[===================>] 307.16M  23.8MB/s    in 15s     

2021-11-30 06:53:40 (20.5 MB/s) - ‘BERTweet_base_transformers.tar.gz.9’ saved [322076118/322076118]

BERTweet_base_transformers/
BERTweet_base_transformers/config.json
BERTweet_base_transformers/bpe.codes
BERTweet_base_transformers/model.bin
BERTweet_base_transformers/dict.txt


In [ ]:
model = BERTweetModelForClassification()
epochs = 7
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 1e-4,
                  eps = 1e-4)

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 1,
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import sys
def validate(model,validation_dataloader, val_labels):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    preds = []
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    t0 = time.time()
    
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            loss, logits = model(b_input_ids,
                                 attention_mask=b_input_mask,
                                 labels=b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        preds.append(logits)
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    
    avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
    print("  Accuracy: {0:.3f} %".format(avg_val_accuracy*100))
    avg_val_loss = total_eval_loss / len(test_dataloader)
    print("  Test Loss: {0:.3f}".format(avg_val_loss))
    
    scores = calculate_scores(preds, val_labels)
    print("  Confusion Matrix: \n{:}".format(scores['confusion_matrix']))
    print("  Precision Score: {0:.3f} %".format(scores['precision_score']*100))
    print("  Recall Score: {0:.3f} %".format(scores['recall_score']*100))
    print("  F1 Score: {0:.3f} %".format(scores['f1_score']*100))

    if avg_val_accuracy*100 >= 75 and scores['f1_score']*100 >= 74:
      torch.save(model.cpu().state_dict(),"ModelEE4.pth")

      model = BERTweetModelForClassification()
      model.load_state_dict(torch.load("ModelEE4.pth"))
      torch.save(model, "Model_EE4_fix.pth")

      sys.exit("Target Acquired. Stopping training process...")

    
    return preds, avg_val_accuracy, avg_val_loss

In [ ]:
def train(model, optimizer, scheduler, train_dataloader, validation_dataloader, val_labels, epochs):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    model.to(device)
    
    training_stats = []
    total_t0 = time.time()

    for epoch_i in range(0, epochs):

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training:')
        
        t0 = time.time()
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad() 
                   
            loss, logits = model(b_input_ids,
                                 attention_mask=b_input_mask,
                                 labels=b_labels)
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)

        print("")
        print("  Training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
            
        _, avg_val_accuracy, avg_val_loss = validate(model,validation_dataloader, val_labels)
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
train(model,optimizer,scheduler,train_dataloader,validation_dataloader, dev_df.label.astype(int), epochs)


======== Epoch 1 / 7 ========
Training:
  Batch    40  of    245.    Elapsed: 0:00:09.
  Batch    80  of    245.    Elapsed: 0:00:18.
  Batch   120  of    245.    Elapsed: 0:00:26.
  Batch   160  of    245.    Elapsed: 0:00:35.
  Batch   200  of    245.    Elapsed: 0:00:44.
  Batch   240  of    245.    Elapsed: 0:00:53.

  Training loss: 1.18
  Training epoch took: 0:00:54
  Accuracy: 60.081 %
  Test Loss: 0.920
  Confusion Matrix: 
[[73  2  0  0  0]
 [57 28  0  1  0]
 [ 2  2  0  6  0]
 [ 4 13  0 45  0]
 [ 0  1  0 11  0]]
  Precision Score: 37.195 %
  Recall Score: 40.494 %
  F1 Score: 36.724 %

======== Epoch 2 / 7 ========
Training:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Batch    40  of    245.    Elapsed: 0:00:09.
  Batch    80  of    245.    Elapsed: 0:00:18.
  Batch   120  of    245.    Elapsed: 0:00:27.
  Batch   160  of    245.    Elapsed: 0:00:35.
  Batch   200  of    245.    Elapsed: 0:00:44.
  Batch   240  of    245.    Elapsed: 0:00:53.

  Training loss: 0.78
  Training epoch took: 0:00:54
  Accuracy: 70.968 %
  Test Loss: 0.679
  Confusion Matrix: 
[[56 19  0  0  0]
 [21 57  0  6  2]
 [ 1  3  0  1  5]
 [ 2  7  0 53  0]
 [ 0  0  0  2 10]]
  Precision Score: 56.117 %
  Recall Score: 61.953 %
  F1 Score: 58.597 %

======== Epoch 3 / 7 ========
Training:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  Batch    40  of    245.    Elapsed: 0:00:09.
  Batch    80  of    245.    Elapsed: 0:00:18.
  Batch   120  of    245.    Elapsed: 0:00:27.
  Batch   160  of    245.    Elapsed: 0:00:35.
  Batch   200  of    245.    Elapsed: 0:00:44.
  Batch   240  of    245.    Elapsed: 0:00:53.

  Training loss: 0.60
  Training epoch took: 0:00:54
  Accuracy: 69.355 %
  Test Loss: 0.953
  Confusion Matrix: 
[[49 25  1  0  0]
 [14 58  2 10  2]
 [ 0  2  1  1  6]
 [ 2  5  0 55  0]
 [ 0  0  1  2  9]]
  Precision Score: 58.731 %
  Recall Score: 61.297 %
  F1 Score: 59.185 %

======== Epoch 4 / 7 ========
Training:
  Batch    40  of    245.    Elapsed: 0:00:09.
  Batch    80  of    245.    Elapsed: 0:00:18.
  Batch   120  of    245.    Elapsed: 0:00:27.
  Batch   160  of    245.    Elapsed: 0:00:35.
  Batch   200  of    245.    Elapsed: 0:00:44.
  Batch   240  of    245.    Elapsed: 0:00:53.

  Training loss: 0.37
  Training epoch took: 0:00:54
  Accuracy: 70.161 %
  Test Loss: 1.500
  Confusion Matrix: 


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
